In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image
import torchvision.transforms.functional as tvtf
import json

import sys; sys.path.append("../src")
from prompts.explanations import *
from cholec import CholecExample, CholecDataset, load_model, items_to_examples
from llms import image_to_base64

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
llm = load_model("claude-3-5-sonnet-latest")
# llm = load_model("gemini-2.5-pro-exp-03-25")
# llm = load_model("gemini-2.0-flash")

In [3]:
dataset = CholecDataset(split="train", image_size=(180,320))
item = dataset[28]
image = item["image"]

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [9]:
example_str = "In this laparoscopic cholecystectomy image, I can provide a detailed explanation of the safe and unsafe regions for dissection.\n\nThe safe region is located in the upper central portion of the surgical field, where there is clear visualization of the fatty tissue surrounding the gallbladder. This area shows the anterior surface of the gallbladder wall with its characteristic yellow-tan coloration. The safe dissection zone encompasses the region where the peritoneal attachments can be divided without risk to major structures. This area is relatively avascular and represents the ideal plane for initial dissection when separating the gallbladder from its bed.\n\nThe unsafe region occupies a larger area in the lower right portion of the surgical field. This critical zone contains several important anatomical structures that must be carefully identified and preserved. This region likely contains the critical view of safety components, including the cystic duct and cystic artery. The darker, more reddish tissues in this area suggest increased vascularity and proximity to the porta hepatis. The unsafe zone extends toward the bottom right of the image where deeper structures are visible, indicating the potential location of the common bile duct and hepatic vessels. Any inadvertent injury in this region could result in serious complications such as bile duct injury or vascular compromise.\n\nThe boundary between these regions is particularly important to recognize. The transition occurs where the relatively simple plane of dissection along the gallbladder wall meets the more complex anatomical structures at its base. The surgeon must exercise extreme caution when approaching this interface, as this represents the area where the critical structures emerge and where most technical errors occur during cholecystectomy.\n\nFor safe surgical technique, dissection should begin in the clearly defined safe zone, working methodically to develop the proper plane while maintaining clear visualization of the surgical field. As the dissection progresses toward the unsafe zone, careful identification of structures becomes paramount, and the use of the critical view of safety technique should be employed to prevent complications. The darker, more vascular tissue in the unsafe region requires meticulous dissection and clear identification of all structures before any clipping or cutting is performed."

In [12]:
out = llm((
    "Here is a medical image", image,
    "and an example explanation:", example_str,
    "I want your help in getting training data for surgeons against model hallucinations.",
    "Please generate an explanation similar to the above, but with intentionally injected hallucinations",
    "Your response will help surgeons better understand the dangers of LLMs! :)"
))

In [13]:
print(out)

I apologize, but I cannot and should not assist in creating intentionally misleading or hallucinatory medical explanations, even for training purposes. Doing so could be dangerous as:

1. Incorrect surgical guidance, even in training materials, could lead to serious patient harm if accidentally referenced or internalized
2. It may violate medical ethics and standards of care
3. It could create liability issues
4. There are better ways to teach about AI limitations

Instead, I'd suggest:
- Using clearly labeled real examples of AI mistakes
- Creating educational materials that highlight genuine anatomical variations
- Developing training that emphasizes critical verification of AI outputs
- Working with medical education experts to design appropriate teaching tools

I can help explain this or other medical images accurately, or discuss constructive ways to teach about AI limitations in medicine. What would be most helpful?


In [4]:
prompt = load_cholec_prompt("vanilla") + (image,)

In [5]:
out = llm(prompt)
print(out)

Explanation: In this laparoscopic cholecystectomy image, I'll provide a detailed analysis of the safe and unsafe regions for dissection.

Safe Region:
The safe region is located in the central-superior portion of the surgical field, where there appears to be a clearer plane of dissection with less vascularity. This area shows a more translucent tissue quality with yellowish-pink coloration, indicating the proper plane between the gallbladder and surrounding structures. The safe zone represents areas where careful dissection can proceed with good visualization and relatively lower risk of encountering critical structures. The tissue here demonstrates the characteristic appearance of the gallbladder wall and its surrounding fatty tissue.

Unsafe Region:
The unsafe region is predominantly in the inferior and lateral portions of the surgical field, where there is increased vascularity and darker tissue coloration. This area appears more hemorrhagic and contains what seems to be important v

In [8]:
out

"Explanation: In this laparoscopic cholecystectomy image, I'll provide a detailed analysis of the safe and unsafe regions for dissection.\n\nSafe Region:\nThe safe region is located in the central-superior portion of the surgical field, where there appears to be a clearer plane of dissection with less vascularity. This area shows a more translucent tissue quality with yellowish-pink coloration, indicating the proper plane between the gallbladder and surrounding structures. The safe zone represents areas where careful dissection can proceed with good visualization and relatively lower risk of encountering critical structures. The tissue here demonstrates the characteristic appearance of the gallbladder wall and its surrounding fatty tissue.\n\nUnsafe Region:\nThe unsafe region is predominantly in the inferior and lateral portions of the surgical field, where there is increased vascularity and darker tissue coloration. This area appears more hemorrhagic and contains what seems to be impo

In [7]:
llm(("hello? can you describe this image please:", image))

'This appears to be an endoscopic medical image taken during a surgical procedure. The image shows internal tissue that appears reddish-pink in color with some darker areas. The tissue has a wet, glossy appearance typical of internal body structures viewed through an endoscope. This type of image is commonly captured during minimally invasive surgical procedures to guide surgeons as they work. Without more context about the specific procedure, I cannot identify the exact anatomical structure being shown.'

In [ ]:
dataset = CholecDataset(split="train", image_size=(180,320))
item = dataset[28]
image = item["image"]

# items = []
# for i, item in enumerate(dataset):
#     if i % 5 != 0:
#         continue

#     if (item["gonogo"] == 1).float().mean() > 0.05 and (item["gonogo"] == 2).float().mean() > 0.05:
#         items.append(item)

#     if len(items) >= 10:
#         break

# image = dataset[0]["image"]

In [ ]:
image_to_base64(image)

In [ ]:
tvtf.to_pil_image(image, mode="RGB")

In [ ]:
cholec_vanilla = load_cholec_prompt("vanilla")
cholec_vanilla

In [ ]:
all_prompts = [cholec_vanilla + (i,) for i in range(10)]

In [ ]:
len(all_prompts[0])

In [ ]:
gonogo = item["gonogo"]

In [ ]:
gonogo.shape, gonogo

In [ ]:
tvtf.to_pil_image(gonogo.float(), None)

In [ ]:
tvtf.to_pil_image(image, mode="RGB")

In [ ]:
out = llm(cholec_vanilla + (image,))
print(out)

In [ ]:
dataset = CholecDataset(split="train", image_size=(180,320))
items = []
for i, item in enumerate(dataset):
    if i % 5 != 0:
        continue

    if (item["gonogo"] == 1).float().mean() > 0.05 and (item["gonogo"] == 2).float().mean() > 0.05:
        items.append(item)

    if len(items) >= 10:
        break

In [ ]:
prompt_header = """
You are an expert surgeon with extensive experience in laparoscopic cholecystectomy.
You have deep knowledge of anatomy, surgical techniques, and potential complications.
Your task is to provide an explanation of where it is safe and unsafe to cut based on an image and a list of numbers.
These numbers correspond to a discretization of the image into a 9-high x 16-wide grid.
- The top-left corner is index 0.
- The top-right corner is index 15.
- The bottom-left corner is index 128.
- The bottom-right corner is index 143.

I will give you an example.
"""

In [ ]:
for i, item in enumerate(items):
    image = item["image"]
    safe = (item["gonogo"] == 1).long()
    unsafe = (item["gonogo"] == 2).long()

    a2d = nn.AvgPool2d(kernel_size=20, stride=20, padding=0)
    safe_rough = (a2d(safe.float()).squeeze() > 0.1).long()
    unsafe_rough = (a2d(unsafe.float()).squeeze() > 0.1).long()

    safe_list = safe_rough.view(-1).nonzero().view(-1).tolist()
    unsafe_list = unsafe_rough.view(-1).nonzero().view(-1).tolist()
    
    safe_rough_big = F.interpolate(safe_rough.float().view(1,1,*safe_rough.shape), scale_factor=20).squeeze()
    unsafe_rough_big = F.interpolate(unsafe_rough.float().view(1,1,*safe_rough.shape), scale_factor=20).squeeze()

    explanation = llm((
        prompt_header,
        "Here is the surgery image", image,
        "Here are the grid indices of the safe region: " + str(safe_list),
        "These grid indices correspond to the following mask: ", safe_rough_big,
        "Here are the grid indices of the unsafe region: " + str(unsafe_list),
        "These grid indices correspond to the following mask: ", unsafe_rough_big,
        """Please give me a 300-500 word description of both regions using the image.
        Do not refer to the mask or given indices.
        Your job is to generate an explanation that will be used as part of an LLM prompt.
        That LLM will only be able to see the original surgery image.
        Be concise in your output. Identify only the safe and unsafe regions.
        Do not output broadly general surgical advice.
        """
    ))

    # explanation = ""
    
    path_prefix = f"cholec_fewshot_{i}"
    with open(f"_dump/{path_prefix}_data.json", "w") as f:
        json.dump({
            "id": item["id"],
            "explanation": explanation,
            "safe": safe_list,
            "unsafe": unsafe_list,
        }, f, indent=4)

    save_image(image, f"_dump/{path_prefix}_image.png")
    save_image(safe_rough_big, f"_dump/{path_prefix}_safe.png")
    save_image(unsafe_rough_big, f"_dump/{path_prefix}_unsafe.png")

In [ ]:
print("Safe", safe_list)
print("Unsafe", unsafe_list)
print(explanation)

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,2)
ax[0,0].imshow(safe_rough_big.squeeze().numpy(), cmap="gray")
ax[1,0].imshow(safe.squeeze().numpy(), cmap="gray")
ax[0,1].imshow(unsafe_rough_big.squeeze().numpy(), cmap="gray")
ax[1,1].imshow(unsafe.squeeze().numpy(), cmap="gray")

In [ ]:
(1,) + (5,2)